In [23]:
import dgl.nn as dglnn
from dgl import from_networkx
import torch.nn as nn
import torch as th
from dgl.nn import SAGEConv
import torch.nn.functional as F
import dgl.function as fn
import networkx as nx
import pandas as pd
import socket
import struct
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [35]:
data = pd.read_csv('/data/sampled_data.csv')

In [38]:
data

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,label
0,1.170.173.114:60943,172.31.65.77:445,-0.275191,-0.154132,-0.066507,-0.020646,-0.051592,-0.056186,-0.369968,-0.247238,1
1,104.86.80.239:443,172.31.67.62:52001,-0.275191,-0.154132,-0.053266,-0.020646,-0.050126,-0.056186,0.396931,-0.247238,1
2,104.90.38.79:443,172.31.67.62:57856,-0.275191,-0.154132,-0.052191,-0.020646,-0.050289,-0.056186,0.396931,-0.247238,1
3,110.78.184.127:54134,172.31.69.25:445,-0.275191,-0.154132,-0.066575,-0.020646,-0.051755,-0.056186,1.390221,-0.247238,1
4,122.226.151.2:23737,172.31.67.62:445,-0.275191,-0.154132,-0.063313,-0.020646,-0.050126,-0.056186,-1.141344,-0.247238,1
...,...,...,...,...,...,...,...,...,...,...,...
994,74.125.28.94:443,172.31.67.62:51431,-0.275191,-0.154132,-0.054693,-0.020646,-0.050941,-0.056186,-1.141344,-0.247238,1
995,80.241.219.91:56717,172.31.69.9:3389,-0.275191,-0.154132,-0.063125,0.295782,-0.050615,1.530418,-1.568346,4.046571,3
996,87.230.72.8:64294,172.31.65.77:3389,-0.275191,-0.154132,-0.063262,-0.020646,-0.050452,-0.056186,-1.887340,-0.247238,1
997,89.39.119.22:58306,172.31.67.62:3389,-0.275191,-0.154132,-0.063272,-0.020646,-0.050778,-0.056186,-1.887340,-0.247238,1


In [36]:
data =data.groupby(['IPV4_SRC_ADDR', 'IPV4_DST_ADDR']).agg({
    'PROTOCOL': 'mean',
    'L7_PROTO': 'mean',
    'IN_BYTES': 'mean',
    'OUT_BYTES': 'mean',
    'IN_PKTS': 'mean',
    'OUT_PKTS': 'mean',
    'TCP_FLAGS': 'mean', 
    'FLOW_DURATION_MILLISECONDS': 'mean',
    'label': 'first' 
}).reset_index()

In [39]:
count_records = data.groupby(['IPV4_SRC_ADDR', 'IPV4_DST_ADDR']).size().reset_index(name='COUNT')

# Hiển thị kết quả
print(count_records)

            IPV4_SRC_ADDR       IPV4_DST_ADDR  COUNT
0     1.170.173.114:60943    172.31.65.77:445      1
1       104.86.80.239:443  172.31.67.62:52001      1
2        104.90.38.79:443  172.31.67.62:57856      1
3    110.78.184.127:54134    172.31.69.25:445      1
4     122.226.151.2:23737    172.31.67.62:445      1
..                    ...                 ...    ...
994      74.125.28.94:443  172.31.67.62:51431      1
995   80.241.219.91:56717    172.31.69.9:3389      1
996     87.230.72.8:64294   172.31.65.77:3389      1
997    89.39.119.22:58306   172.31.67.62:3389      1
998    91.216.195.218:443  172.31.67.62:50194      1

[999 rows x 3 columns]


In [4]:
G = nx.from_pandas_edgelist(
    data,
    source='IPV4_SRC_ADDR',
    target='IPV4_DST_ADDR',
    edge_attr=['PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 
                                'IN_PKTS', 'OUT_PKTS', 'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'label'],
    create_using=nx.MultiDiGraph()
)

In [10]:
le = LabelEncoder()
data['IPV4_SRC_ADDR'] = le.fit_transform(data['IPV4_SRC_ADDR'])
data['IPV4_DST_ADDR'] = le.fit_transform(data['IPV4_DST_ADDR'])


In [11]:
data

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,label
0,334,186,-0.275191,-0.154132,-0.066702,-0.020646,-0.051918,-0.056186,-0.561980,-0.247238,0
1,212,425,-0.275191,-0.154132,-0.066731,-0.020646,-0.051918,-0.056186,-0.561980,-0.247238,1
2,560,186,-0.275191,-0.154132,-0.066263,-0.020646,-0.051266,-0.056186,-1.887340,-0.247238,0
3,212,339,-0.275191,-0.154132,-0.066731,-0.020646,-0.051918,-0.056186,-0.561980,-0.247238,1
4,212,410,-0.275191,-0.154132,-0.066731,-0.020646,-0.051918,-0.056186,-0.561980,-0.247238,1
...,...,...,...,...,...,...,...,...,...,...,...
995,21,182,-0.275191,-0.154132,-0.059110,-0.020646,-0.048171,-0.056186,0.396931,-0.247238,2
996,350,186,-0.275191,-0.154132,-0.065118,-0.020646,-0.051266,-0.056186,-1.568346,-0.247238,0
997,49,182,-0.275191,-0.154132,-0.058857,-0.020646,-0.047846,-0.056186,0.396931,-0.247238,2
998,212,431,-0.275191,-0.154132,-0.066731,-0.020646,-0.051918,-0.056186,-0.561980,-0.247238,1


In [12]:
features = data[['PROTOCOL','L7_PROTO','IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS', 'FLOW_DURATION_MILLISECONDS']].values

In [15]:
import dgl
src_nodes = data['IPV4_SRC_ADDR'].values
dst_nodes = data['IPV4_DST_ADDR'].values

# Tạo đồ thị từ các chỉ số nút
g = dgl.graph((src_nodes, dst_nodes))

In [16]:
labels = data['label'].values

In [17]:
labels

array([0, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 0, 0, 1, 1, 2, 1, 1, 1, 0, 0,
       0, 1, 1, 2, 0, 1, 0, 1, 2, 2, 1, 2, 2, 2, 0, 0, 2, 1, 0, 1, 1, 1,
       1, 1, 3, 2, 2, 0, 1, 0, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 0, 1, 2, 1,
       1, 2, 1, 1, 2, 1, 3, 3, 1, 1, 3, 2, 1, 1, 1, 1, 3, 1, 2, 0, 2, 1,
       3, 2, 1, 2, 2, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 1, 2, 2, 0, 1, 2, 0,
       1, 2, 1, 2, 1, 0, 2, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1,
       0, 2, 1, 1, 2, 3, 0, 0, 1, 1, 0, 3, 0, 1, 0, 2, 1, 1, 0, 1, 1, 1,
       2, 1, 1, 1, 2, 1, 1, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2,
       0, 2, 1, 1, 1, 3, 0, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0, 1, 1, 2, 1,
       2, 0, 2, 2, 0, 2, 2, 1, 2, 0, 1, 1, 2, 1, 1, 1, 2, 2, 0, 1, 1, 1,
       1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 0, 1, 3, 2, 2, 0,
       1, 2, 2, 1, 2, 3, 1, 1, 2, 1, 2, 2, 0, 1, 2, 3, 1, 1, 1, 1, 1, 2,
       3, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [18]:
train_mask, test_mask = train_test_split(np.arange(len(labels)), test_size=0.2, random_state=42)
train_mask = torch.tensor(train_mask)
test_mask = torch.tensor(test_mask)


In [21]:
class GraphSAGEModel(torch.nn.Module):
    def __init__(self, in_feats, hidden_size, out_feats):
        super(GraphSAGEModel, self).__init__()
        self.conv1 = SAGEConv(in_feats, hidden_size, aggregator_type='mean')
        self.conv2 = SAGEConv(hidden_size, out_feats, aggregator_type='mean')

    def forward(self, g, features):
        h = self.conv1(g, features)
        h = torch.relu(h)
        h = self.conv2(g, h)
        return h

In [25]:
features.shape[1]

7

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = GraphSAGEModel(in_feats=features.shape[1], hidden_size=16, out_feats=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCEWithLogitsLoss()

# Chuyển đổi dữ liệu sang tensor
g = g.to(device)
features = torch.tensor(features, dtype=torch.float32).to(device)
labels = torch.tensor(labels, dtype=torch.float32).to(device)

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    logits = model(g, features)  # Tính toán đầu ra
    loss = loss_fn(logits[train_mask], labels[train_mask].unsqueeze(1))  # Tính toán loss
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# 6. Dự đoán cho các cạnh
model.eval()
with torch.no_grad():
    edge_logits = model(g, features)
    predictions = torch.sigmoid(edge_logits).cpu().numpy()
    predicted_labels = (predictions > 0.5).astype(int)

# 7. Đánh giá mô hình
from sklearn.metrics import accuracy_score

test_preds = predicted_labels[test_mask]
test_labels = labels[test_mask].cpu().numpy()
accuracy = accuracy_score(test_labels, test_preds)
print(f'Accuracy: {accuracy * 100:.2f}%')

DGLError: Expect number of features to match number of nodes (len(u)). Got 1000 and 581 instead.